In [94]:
from elements.batteryModel import LithiumIonBattery

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from mat4py import loadmat
from scipy.linalg import block_diag, cholesky
from copy import copy


In [95]:
LIB1 = LithiumIonBattery('models/PANmodel.mat', T=25, dt=1)
i = 1
newState = LIB1.stateEqn(i)
voltage = LIB1.outputEqn(i)
# LIB1.updateState(newState)

newState, voltage

((array([[0.99998884]]), array([[0.23923906]]), array([[-0.00182312]])),
 array([[4.09849743]]))

In [96]:
LIB1.outputEqn(0)

array([[4.09963424]])

In [97]:
'''
# SPKF Step 1: State estimate time update
# 1a: Calculate augmented state estimate, including ...
xhata = np.array([xhat.item(), 0, 0])  # process and sensor noise mean
# 1b: Get desired Cholesky factor
Pxa = block_diag(SigmaX,SigmaW,SigmaV)
sPxa = cholesky(Pxa, lower=True)    
# 1c: Calculate sigma points (strange indexing of xhat to avoid
X = xhata.reshape(3,1) + h*np.hstack([np.zeros((Nxa, 1)), sPxa, -sPxa])
# 1d: Calculate state equation for every element
Xx = np.sqrt(5+X[0,:]) + X[1,:]
xhat = Wmx@Xx
'''
SigmaX = block_diag(1e2,1e-2,1e-3)  # uncertainty of initial state
SigmaV = 3e-1  # Uncertainty of voltage sensor, output equation
SigmaW = 4e0   # Uncertainty of current sensor, state equation
Nx = 3
Nw = 1
Nv = 1
Na = Nx+Nw+Nv

Wmx = np.zeros(2*Na+1)
h = np.sqrt(3)
Wmx[0] = (h**2-Na)/(h**2)
Wmx[1:] = 1/(2*h**2) 
Wcx=Wmx

priorI = 10
ik=10
vk = 4
Qbump = 5

# Step 1a: State estimate time update
#          - Create xhatminus augmented SigmaX points
#          - Extract xhatminus state SigmaX points
#          - Compute weighted average xhatminus(k)

xhat = np.vstack(newState)

# Step 1a-1: Create augmented xhat and SigmaX
sigmaXa = block_diag(SigmaX,SigmaW,SigmaV)
sigmaXa = np.real(cholesky(sigmaXa, lower=True))

# sigmaXa=[real(sigmaXa) zeros([Nx Nw+Nv]); zeros([Nw+Nv Nx]) Snoise]
# xhata = np.array([xhat.item(), 0, 0])
xhata = np.vstack([xhat, np.zeros((Nw+Nv,1))])

#   spkfData.Snoise = real(chol(diag([SigmaW; SigmaV]),'lower'));

# Step 1a-2: Calculate SigmaX points
Xa = xhata.reshape(Na,1) + h*np.hstack([np.zeros((Na, 1)), sigmaXa, -sigmaXa])
# Step 1a-3: Time update from last iteration until now
#     stateEqn(xold,current,xnoise)
Xx = LIB1.stateEqn(priorI, Xa[Nx:Nx+Nw,:], Xa[:Nx,:])
Xx = np.vstack(Xx)
xhat = np.sum(Wmx*Xx, axis=-1, keepdims=True)


# Step 1b: Error covariance time update
#          - Compute weighted covariance sigmaminus(k)
#            (strange indexing of xhat to avoid "repmat" call)

SigmaX = np.sum(Wcx*(Xx - xhat)**2, axis=-1)

# Step 1c: Output estimate
#          - Compute weighted output estimate yhat(k)
Y = LIB1.outputEqn(ik, Xa[Nx+Nw:,:], Xx)
yhat = np.sum(Wcx*Y, axis=-1)

# Step 2a: Estimator gain matrix
SigmaXY = (Xx - xhat)@np.diag(Wcx)@(Y - yhat).T
SigmaY  = (Y - yhat)@np.diag(Wcx)@(Y - yhat).T
L = SigmaXY/SigmaY

# Step 2b: State estimate measurement update
r = vk - yhat  # residual.  Use to check for sensor errors...
if r**2 > 100*SigmaY: L=0
xhat = xhat + L*r 
xhat[0] = np.clip(xhat[0], -0.05, 1.05)
xhat[-1] = np.clip(xhat[-1], -1, 1)

# Step 2c: Error covariance measurement update
SigmaX = SigmaX - L*SigmaY*L.T
_,S,V = np.linalg.svd(SigmaX)
HH = V*S*V.T
SigmaX = (SigmaX + SigmaX.T + HH + HH.T)/4 # Help maintain robustness

# Q-bump code
if r**2>4*SigmaY: # bad voltage estimate by 2-SigmaX, bump Q 
    printf('Bumping sigmax\n')
    SigmaX[0,0] = SigmaX[0,0]*Qbump


In [12]:
SigmaX.shape[0], SigmaV.shape, SigmaW.shape

AttributeError: 'float' object has no attribute 'shape'

In [121]:
class SPKF:
    def __init__(self, target_model, SigmaX0, SigmaV, SigmaW):
        # store model
        self.model = copy(target_model)
        ir0   = 0                        
        hk0   = 0                        
        SOC0  = 1 # self.model.OCVfromSOC(self.model.outputEqn(ir0))
        
        self.xhat  = np.vstack([SOC0,ir0,hk0]) # initial state

        # Covariance values
        self.SigmaX = SigmaX0
        self.SigmaV = SigmaV
        self.SigmaW = SigmaW

        # SPKF specific parameters
        self.Nx = SigmaX0.shape[0]
        self.Nw = 1
        self.Nv = 1
        self.Na = self.Nx+self.Nw+self.Nv

        Wmx = np.zeros(2*self.Na+1)
        self.h = np.sqrt(3)
        Wmx[0] = (self.h**2-self.Na)/(self.h**2)
        Wmx[1:] = 1/(2*self.h**2) 
        Wcx=Wmx
        self.Wm = Wmx.reshape(-1,1) # mean
        self.Wc = Wcx.reshape(-1,1) # covar
        self.Qbump = 5

        # previous value of current
        self.priorI = 0
        
    def iter(self, ik, vk):
        # Step 1a-1: Create augmented xhat and SigmaX
        eigval, _ = np.linalg.eig(self.SigmaX)
        # print(eigval)
        sigmaXa = block_diag(self.SigmaX, self.SigmaW, self.SigmaV)
        sigmaXa = np.real(cholesky(sigmaXa, lower=True))
        xhata = np.vstack([self.xhat, np.zeros((self.Nw+self.Nv,1))])

        # Step 1a-2: Calculate SigmaX points
        Xa = xhata.reshape(self.Na,1) + self.h*np.hstack([np.zeros((self.Na, 1)), sigmaXa, -sigmaXa])

        # Step 1a-3: Time update from last iteration until now
        Xx = self.model.stateEqn(self.priorI, Xa[self.Nx:self.Nx+self.Nw,:], Xa[:self.Nx,:])
        Xx = np.vstack(Xx)
        xhat = Xx@self.Wm

        # Step 1b: Error covariance time update
        #          - Compute weighted covariance sigmaminus(k)
        #            (strange indexing of xhat to avoid "repmat" call)
        SigmaX = (Xx - xhat)@np.diag(self.Wm.ravel())@(Xx - xhat).T

        # Step 1c: Output estimate
        #          - Compute weighted output estimate yhat(k)
        Y = self.model.outputEqn(ik, Xa[self.Nx+self.Nw:,:], Xx)
        yhat = Y@self.Wc

        # Step 2a: Estimator gain matrix
        SigmaXY = (Xx - xhat)@np.diag(self.Wc.ravel())@(Y - yhat).T
        SigmaY  = (Y - yhat)@np.diag(self.Wc.ravel())@(Y - yhat).T
        L = SigmaXY/SigmaY

        # Step 2b: State estimate measurement update
        r = vk - yhat  # residual.  Use to check for sensor errors...
        if r**2 > 100*SigmaY: L=0
        # print(L, r, yhat, vk)
        xhat = xhat + L*r 
        xhat[0] = np.clip(xhat[0], -0.05, 1.05)
        xhat[-1] = np.clip(xhat[-1], -1, 1)

        # Step 2c: Error covariance measurement update
        SigmaX = SigmaX - L*SigmaY*L.T
        # eigval, _ = np.linalg.eig(SigmaX)
        # while not np.prod(eigval>=0):
        #     _,S,V = np.linalg.svd(SigmaX)
        #     HH = V@np.diag(S)@V.T
        #     SigmaX = (SigmaX + SigmaX.T + HH + HH.T)/4 # Help maintain robustness
        #     eigval, _ = np.linalg.eig(SigmaX)
        
        Y = (SigmaX + SigmaX.T)/2
        eigval, eigvec = np.linalg.eig(Y)
        eigvalp = np.maximum(eigval,0) + 1e-10
        SigmaX = eigvec@np.diag(eigvalp)@eigvec.T

        # Q-bump code
        if r**2>4*SigmaY: # bad voltage estimate by 2-SigmaX, bump Q 
            printf('Bumping sigmax\n')
            SigmaX[0,0] = SigmaX[0,0]*self.Qbump
        
        # Save data for next iteration...
        self.priorI = ik
        self.SigmaX = SigmaX
        self.xhat = xhat
        
        zk = self.xhat[0]
        zkbnd = 3*np.sqrt(SigmaX[0,0])
        return zk,zkbnd
        
SigmaX = block_diag(1e2,1e-2,1e-3)  # uncertainty of initial state
SigmaV = 3e-1  # Uncertainty of voltage sensor, output equation
SigmaW = 4e0   # Uncertainty of current sensor, state equation
LIB1_SPKF = SPKF(LIB1, SigmaX, SigmaV, SigmaW)

In [122]:
LIB1_SPKF.Wm.shape

(11, 1)

In [123]:
LIB1_SPKF.iter(5, 4)

(array([0.96726781]), 0.3865585692149507)

In [ ]:

% Step 1a: State estimate time update
%          - Create xhatminus augmented SigmaX points
%          - Extract xhatminus state SigmaX points
%          - Compute weighted average xhatminus(k)

% Step 1a-1: Create augmented SigmaX and xhat
[sigmaXa,p] = chol(SigmaX,'lower'); 
if p>0,
fprintf('Cholesky error.  Recovering...\n');
theAbsDiag = abs(diag(SigmaX));
sigmaXa = diag(max(SQRT(theAbsDiag),SQRT(spkfData.SigmaW)));
end
sigmaXa=[real(sigmaXa) zeros([Nx Nw+Nv]); zeros([Nw+Nv Nx]) Snoise];
xhata = [xhat; zeros([Nw+Nv 1])];
% NOTE: sigmaXa is lower-triangular

% Step 1a-2: Calculate SigmaX points (strange indexing of xhata to 
% avoid "repmat" call, which is very inefficient in MATLAB)
Xa = xhata(:,ones([1 2*Na+1])) + ...
    spkfData.h*[zeros([Na 1]), sigmaXa, -sigmaXa];

% Step 1a-3: Time update from last iteration until now
%     stateEqn(xold,current,xnoise)
Xx = stateEqn(Xa(1:Nx,:),I,Xa(Nx+1:Nx+Nw,:)); 
xhat = Xx*spkfData.Wm;

% Step 1b: Error covariance time update
%          - Compute weighted covariance sigmaminus(k)
%            (strange indexing of xhat to avoid "repmat" call)
Xs = Xx - xhat(:,ones([1 2*Na+1]));
SigmaX = Xs*diag(Wc)*Xs';

% Step 1c: Output estimate
%          - Compute weighted output estimate yhat(k)
I = ik; yk = vk;
Y = outputEqn(Xx,I,Xa(Nx+Nw+1:end,:),Tk,model);
yhat = Y*spkfData.Wm;

% Step 2a: Estimator gain matrix
Ys = Y - yhat(:,ones([1 2*Na+1]));
SigmaXY = Xs*diag(Wc)*Ys';
SigmaY = Ys*diag(Wc)*Ys';
L = SigmaXY/SigmaY; 

% Step 2b: State estimate measurement update
r = yk - yhat; % residual.  Use to check for sensor errors...
if r^2 > 100*SigmaY, L(:,1)=0.0; end 
xhat = xhat + L*r; 
xhat(zkInd)=min(1.05,max(-0.05,xhat(zkInd)));
xhat(hkInd) = min(1,max(-1,xhat(hkInd)));

% Step 2c: Error covariance measurement update
SigmaX = SigmaX - L*SigmaY*L';
[~,S,V] = svd(SigmaX);
HH = V*S*V';
SigmaX = (SigmaX + SigmaX' + HH + HH')/4; % Help maintain robustness

% Q-bump code
if r^2>4*SigmaY, % bad voltage estimate by 2-SigmaX, bump Q 
fprintf('Bumping sigmax\n');
SigmaX(zkInd,zkInd) = SigmaX(zkInd,zkInd)*spkfData.Qbump;
end

# SVD

In [71]:
import numpy as np
from scipy.linalg import block_diag
SigmaX = np.load('sigmax.npy')
SigmaX

# SigmaX = block_diag(1e2,1e-2,1e-3)

array([[0.08988908, 0.23474708, 0.        ],
       [0.15640708, 0.23472888, 0.        ],
       [0.15638889, 0.23472889, 0.        ]])

In [64]:
eigval, _ = np.linalg.eig(SigmaX)
eigval

array([ 0.        ,  0.36715221, -0.04253424])

In [92]:
Y = (SigmaX + SigmaX.T)/2
eigval, eigvec = np.linalg.eig(Y)
eigvalp = np.maximum(eigval,0)
Yp = eigvec@np.diag(eigvalp)@eigvec.T
np.linalg.eig(Yp)

(array([4.18383173e-01, 4.78772409e-19, 3.01801413e-17]),
 array([[-0.53975366, -0.05919858,  0.83723915],
        [-0.7788889 , -0.34332254, -0.53621971],
        [-0.31937136,  0.93735008, -0.10723351]]))